In [25]:
import hiski_kastetut
import networkx as nx
import matplotlib.pyplot as plt
import time
import numpy as np
import json

In [33]:
reload(hiski_kastetut)
edgelist, xref2hid, G_data = hiski_kastetut.parse_hiski_sour("../../python/pekka_valta/Kuopio_20140308.ged")

3686 sources available
Weak match (sim=0.000): H: - G:((namn sakn)
Weak match (sim=-1.000): H:zacharias - G:maria
Weak match (sim=-1.000): H:walborg - G:carin
Weak match (sim=-1.000): H:nils - G:anna
Weak match (sim=0.000): H: - G:((namn saknas))
Weak match (sim=0.645): H:iacharias - G:zacharias johansson
Weak match (sim=0.517): H:hendrich - G:heikki ollinpoika
Weak match (sim=0.517): H:hendrich - G:heikki ollinpoika
Weak match (sim=0.645): H:iacharias - G:zacharias johansson
Weak match (sim=-1.000): H:peer - G:anna
Weak match (sim=0.000): H: - G:((namn sakn)
Weak match (sim=0.645): H:iacharias - G:zacharias johansson
Found a better match: 3810272->@I7300@ (instead of 3810272->@I6889@)
Weak match (sim=0.678): H:hend. - G:hendrich hind.ss.
Weak match (sim=0.000): H: - G:((namn saknas))
Weak match (sim=-1.000): H:christin - G:anna
Weak match (sim=0.646): H:and. - G:anders sigfr.ss.
Weak match (sim=0.000): H: - G:((namn sakn)
Weak match (sim=0.667): H:påll - G:pål mich.ss.
Weak match (sim

In [34]:
with open("../../python/data/gt_edge_list4.json", "w") as fout:
    json.dump(G_data, fout)

In [5]:
G = nx.Graph(data=edgelist)
ccs = nx.connected_component_subgraphs(G)
ccs = list(ccs)

In [8]:
c_sizes = sorted([CG.order() for CG in ccs])[::-1]
print c_sizes[:30]
print sum(c_sizes)
len(ccs[1].nodes())

[4756, 164, 144, 37, 34, 33, 33, 25, 23, 22, 22, 20, 19, 18, 17, 17, 17, 17, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14]
12700


3

In [13]:
def graph_intersection(G, nodes):
    n_int = len(set(G.nodes()) & set(nodes))
    n_pairs = (n_int*n_int - n_int) / 2
    return n_int, n_pairs

n_int, n_pairs = graph_intersection(ccs[0], xref2hid.keys())
print n_int, n_pairs
print len(xref2hid.keys())

n_ints = []
for gg in ccs:
    n_int, n_pairs = graph_intersection(gg, xref2hid.keys())
    n_ints.append(n_int)
print n_ints[:60]
print sum(n_ints)
n_ints = np.array(n_ints)
print "Non isolated hiski ids:", sum(n_ints[n_ints>1])

344 58996
3118
[344, 0, 3, 2, 1, 5, 1, 1, 5, 0, 1, 63, 1, 1, 2, 0, 2, 1, 1, 2, 0, 9, 2, 1, 5, 1, 2, 1, 8, 1, 0, 3, 0, 0, 3, 3, 0, 18, 5, 0, 4, 0, 1, 11, 1, 1, 0, 0, 0, 3, 56, 2, 6, 0, 0, 0, 5, 2, 2, 0]
3103
Non isolated hiski ids: 2475


In [24]:
# All shortest paths
#paths = nx.floyd_warshall(ccs[1])
#print max(paths.values()[0].values())

def nearby_pairs(G, nodes, max_dist=2):
    nodes = list(set(G.nodes()) & set(nodes))
    #print len(nodes), "nodes"
    #paths = nx.floyd_warshall(G)
    paths = nx.all_pairs_shortest_path_length(G, cutoff=max_dist)
    n_pairs = 0
    for (i, node) in enumerate(nodes):
        for (j, node2) in enumerate(nodes[i+1:]):
            #if paths[node][node2] <= max_dist:
            if node in paths and node2 in paths[node]:
                n_pairs += 1
    return n_pairs


print nearby_pairs(ccs[0], xref2hid.keys(), 1), "pairs"

t0 = time.time()
n_all_pairs = 0
for gg in ccs:
    n_all_pairs += nearby_pairs(gg, xref2hid.keys(), 10)
print "%d pairs in all components." % n_all_pairs
print "Elapsed %.2f seconds." % (time.time()-t0)

53 pairs
25443 pairs in all components.
Elapsed 2.68 seconds.


In [20]:
import hiski_sqlalchemy as db

# Geographic distribution of hiski ids
def parish_counts(hiski_ids):
    counts = {}
    for hid in hiski_ids:
        person = db.Person.query.filter(db.Person.id==hid).first()
        parish = person.parish_id
        counts[parish] = counts.get(parish, 0) + 1
    parish_tuples = sorted(counts.items(), key=lambda tup: tup[1])[::-1]
    return parish_tuples

parish_tuples = parish_counts(xref2hid.values())
for (pid, count) in parish_tuples[:30]:
    print "%d\t%s" % (count, pid)

3024	248
39	317
9	394
8	146
7	174
5	574
5	208
4	252
4	246
3	126
3	112
1	511
1	222
1	154
1	410
1	212
1	206
1	354


In [19]:
a = {'a':3, 'b':3425, 'd':2}
print sorted(a.items(), key=lambda tup: tup[1])

[('d', 2), ('a', 3), ('b', 3425)]
